In [19]:
import argparse
import numpy as np
import pandas as pd
import datetime
import dateutil.parser as time_parser
import os
import time
import warnings
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.amp
import torch.nn as nn
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader 
from torch import optim
from data_provider.data_factory import data_provider
from exp.exp_basic import Exp_Basic
from models import FEDformer, Autoformer, Informer, Transformer
from utils.tools import EarlyStopping, adjust_learning_rate, visual
from utils.metrics import metric
from utils.timefeatures import time_features

import pickle


In [3]:
%load_ext autoreload
%autoreload 2

#TODO
something goes wrong with dimensions 

batch_x: torch.Size([32, 384, 105])

batch_y: torch.Size([32, 192, 105])

batch_x_mark: torch.Size([32, 384, 6])

batch_y_mark: torch.Size([32, 192, 6])

dec_inp: torch.Size([32, 144, 105])

In [24]:
class ForgingData(Dataset):
    def __init__(self, preload_path=None):
        # forecasting task info
        self.seq_len = 24 * 4 * 4
        self.label_len = 24 * 4
        self.pred_len = 24 * 4
        
        if preload_path:
            self.load(preload_path)
        else:
            self.data = pd.read_csv("../../data/timeseries_1month.csv")
            self.data_val = pd.read_csv("../../data/timeseries.csv")
            
            self.data.drop_duplicates(["Received Time"], inplace=True, ignore_index=True)
            self.data.drop(columns=["Unnamed: 0"])
            self.data_val.drop_duplicates(["Received Time"], inplace=True, ignore_index=True)
            self.data_val.drop(columns=["Unnamed: 0"])
            
            # self.data["Received Time"] = self.data["Received Time"].apply(time_parser.parse)
            self.data["Received Time"] = pd.to_datetime(self.data["Received Time"], format="ISO8601")
            self.data = self.data.set_index("Received Time")
            self.data = self.data.resample("100ms").ffill()
            self.data.dropna(inplace=True)
            self.data = self.data.reset_index()
            
            # self.data_val["Received Time"] = self.data_val["Received Time"].apply(time_parser.parse)
            self.data_val["Received Time"] = pd.to_datetime(self.data_val["Received Time"], format="ISO8601")
            self.data_val = self.data_val.set_index("Received Time")
            self.data_val = self.data_val.resample("100ms").ffill()
            self.data_val.dropna(inplace=True)
            self.data_val = self.data_val.reset_index()
            
            # time features encoded secondly
            self.data_stamp = time_features(pd.to_datetime(self.data["Received Time"].values), freq="s")
            self.data_stamp_val = time_features(pd.to_datetime(self.data_val["Received Time"].values), freq="s")
            self.data_stamp = self.data_stamp.transpose(1, 0)
            self.data_stamp_val = self.data_stamp_val.transpose(1, 0)
            
            self.filter = set()
            with open("/home/malio/EPICMAP/src/opcua/data_analysis/useless_counters.txt", "r") as f:
                to_filter = f.readline()
                while to_filter:
                    self.filter.add(to_filter.rstrip())
                    to_filter = f.readline()
            
            self.data = self.data.drop(columns=self.filter, errors="ignore")
            self.data_val = self.data_val.drop(columns=self.filter, errors="ignore")
            
            self.data = self.data.drop(columns=["Received Time"], errors="ignore")
            self.data_val = self.data_val.drop(columns=["Received Time"], errors="ignore")
            
            #scaling
            self.scaler = StandardScaler()
            self.data = self.scaler.fit_transform(self.data)
            self.data_val = self.scaler.transform(self.data_val)
            
            #aliases
            self.data_x = self.data
            self.data_y = self.data
                
    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]

        return seq_x, seq_y, seq_x_mark, seq_y_mark
    
    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1
    
    def save(self, path):
        to_save = {}
        to_save["data"] = self.data
        to_save["data_val"] = self.data_val
        to_save["data_stamp"] = self.data_stamp
        to_save["data_stamp_val"] = self.data_stamp_val
        to_save["filter"] = self.filter
        to_save["scaler"] = self.scaler
        with open(path, "wb") as f:
            pickle.dump(to_save, f, protocol=4)
    
    def load(self, path):
        with open(path, "rb") as f:
            to_load = pickle.load(f)
            self.data = to_load["data"]
            self.data_val = to_load["data_val"]
            self.data_stamp = to_load["data_stamp"]
            self.data_stamp_val = to_load["data_stamp_val"]
            self.filter = to_load["filter"]
            self.scaler = to_load["scaler"]
            self.data_x = self.data
            self.data_y = self.data

# data reader test

In [25]:
forge_data = ForgingData()

In [27]:
preloaded_fdata= ForgingData(preload_path="/home/malio/EPICMAP/src/opcua/data/processed_forging_data.pkl")

In [28]:
data1, val_data1 = train_test_split(forge_data, test_size=0.2, random_state=42)
data2, val_data2 = train_test_split(preloaded_fdata, test_size=0.2, random_state=42)

In [29]:
train_loader1 = DataLoader(data1, batch_size=32, shuffle=False)
val_loader1 = DataLoader(val_data1, batch_size=32, shuffle=False)
train_loader2 = DataLoader(data2, batch_size=32, shuffle=False)
val_loader2 = DataLoader(val_data2, batch_size=32, shuffle=False)

In [30]:
for i, (x, y, x_mark, y_mark) in enumerate(train_loader1):
    print(x.shape, y.shape, x_mark.shape, y_mark.shape)
    break

torch.Size([32, 384, 105]) torch.Size([32, 192, 105]) torch.Size([32, 384, 6]) torch.Size([32, 192, 6])


In [31]:
for i, (x, y, x_mark, y_mark) in enumerate(train_loader2):
    print(x.shape, y.shape, x_mark.shape, y_mark.shape)
    break

torch.Size([32, 384, 105]) torch.Size([32, 192, 105]) torch.Size([32, 384, 6]) torch.Size([32, 192, 6])


In [4]:
data = pd.read_csv("../../data/timeseries_1month.csv")
data_val = pd.read_csv("../../data/timeseries.csv")

# forecasting task info
seq_len = 24 * 4 * 4
label_len = 24 * 4
pred_len = 24 * 4
data.drop_duplicates(["Received Time"], inplace=True, ignore_index=True)
data = data.drop(columns=["Unnamed: 0"])
data_val.drop_duplicates(["Received Time"], inplace=True, ignore_index=True)
data_val = data_val.drop(columns=["Unnamed: 0"])

In [5]:
data["Received Time"] = pd.to_datetime(data["Received Time"], format="ISO8601")
data = data.set_index("Received Time")
data = data.resample("100ms").ffill()
data.dropna(inplace=True)
data = data.reset_index()

data_val["Received Time"] = pd.to_datetime(data_val["Received Time"], format="ISO8601")
data_val = data_val.set_index("Received Time")
data_val = data_val.resample("100ms").ffill()
data_val.dropna(inplace=True)
data_val = data_val.reset_index()

In [6]:
# time features
data_stamp = time_features(pd.to_datetime(data["Received Time"].values), freq="s")
data_stamp_val = time_features(pd.to_datetime(data_val["Received Time"].values), freq="s")
data_stamp = data_stamp.transpose(1, 0)
data_stamp_val = data_stamp_val.transpose(1, 0)

In [7]:
filter = set()
with open("/home/malio/EPICMAP/src/opcua/data_analysis/useless_counters.txt", "r") as f:
    to_filter = f.readline()
    while to_filter:
        filter.add(to_filter.rstrip())
        to_filter = f.readline()

data = data.drop(columns=filter, errors="ignore")
data_val = data_val.drop(columns=filter, errors="ignore")

data = data.drop(columns=["Received Time"], errors="ignore")
data_val = data_val.drop(columns=["Received Time"], errors="ignore")


In [8]:
#scaling
# for testing
scaler = StandardScaler()
data = scaler.fit_transform(data)
data_val = scaler.transform(data_val)
data_x = data
data_y = data

In [15]:
def save(path):
    to_save = {}
    to_save["data"] = data
    to_save["data_val"] = data_val
    to_save["data_stamp"] = data_stamp
    to_save["data_stamp_val"] = data_stamp_val
    to_save["filter"] = filter
    to_save["scaler"] = scaler
    with open(path, "wb") as f:
        pickle.dump(to_save, f, protocol=4)

def load(path):
    with open(path, "rb") as f:
        to_load = pickle.load(f)
        data = to_load["data"]
        data_val = to_load["data_val"]
        data_stamp = to_load["data_stamp"]
        data_stamp_val = to_load["data_stamp_val"]
        filter = to_load["filter"]
        scaler = to_load["scaler"]
        data_x = data
        data_y = data

In [12]:
save("./temp/save-data.pkl")

In [13]:
data_x.shape, data_y.shape, data_stamp.shape, data_stamp_val.shape

((32313748, 104), (32313748, 104), (32313748, 6), (6735215, 6))

In [16]:
load("./temp/save-data.pkl")

In [17]:
data_x.shape, data_y.shape, data_stamp.shape, data_stamp_val.shape

((32313748, 104), (32313748, 104), (32313748, 6), (6735215, 6))

# main blocks

In [21]:
fdata = ForgingData()

In [19]:
fdata.save("../../data/processed_forging_data.pkl")

OverflowError: serializing a string larger than 4 GiB requires pickle protocol 4 or higher

In [22]:
test_loading_data = ForgingData(preload_path="../../data/processed_forging_data.pt")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/processed_forging_data.pt'

In [9]:
args = {
    'is_training': 1,
    'task_id': 'test',
    'model': 'FEDformer',
    'version': 'Fourier',
    'mode_select': 'random',
    'modes': 64,
    'L': 3,
    'base': 'legendre',
    'cross_activation': 'tanh',
    'data': 'ETTh1',
    'root_path': './dataset/ETT/',
    'data_path': 'ETTh1.csv',
    'features': 'M',
    'target': 'OT',
    'freq': 's',
    'detail_freq': 'h',
    'checkpoints': './checkpoints/',
    'seq_len': 96,
    'label_len': 96,
    'pred_len': 96,
    'enc_in': 105,
    'dec_in': 105,
    'c_out': 105,
    'd_model': 512,
    'n_heads': 8,
    'e_layers': 2,
    'd_layers': 1,
    'd_ff': 2048,
    'moving_avg': [12],
    'factor': 1,
    'distil': True,
    'dropout': 0.05,
    'embed': 'timeF',
    'activation': 'gelu',
    'output_attention': False,
    'do_predict': False,
    'num_workers': 10,
    'itr': 3,
    'train_epochs': 10,
    'batch_size': 32,
    'patience': 3,
    'learning_rate': 0.0001,
    'des': 'test',
    'loss': 'mse',
    'lradj': 'type1',
    'use_amp': False,
    'use_gpu': True,
    'gpu': 0,
    'use_multi_gpu': True,
    'devices': '1,2,3'
}

args = argparse.Namespace(**args)

In [10]:
def train(args, train_loader, model, optim, device, criterion, epoch, train_steps, iter_count, scaler=None):
    train_loss = []
    time_now = time.time()
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(train_loader):
        iter_count += 1
        optim.zero_grad()
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_x_mark = batch_x_mark.float().to(device)
        batch_y_mark = batch_y_mark.float().to(device)

        # decoder input
        dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
        dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float().to(device)
        
        # encoder - decoder
        if args.use_amp:
            with torch.amp.autocast("cuda"):
                if args.output_attention:
                    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                else:
                    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                f_dim = -1 if args.features == 'MS' else 0
                batch_y = batch_y[:, -args.pred_len:, f_dim:].to(device)
                loss = criterion(outputs, batch_y)
                train_loss.append(loss.item())
        else:
            if args.output_attention:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
            else:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

            f_dim = -1 if args.features == 'MS' else 0
            batch_y = batch_y[:, -args.pred_len:, f_dim:].to(device)

            loss = criterion(outputs, batch_y)
            train_loss.append(loss.item())

        if (i + 1) % 100 == 0:
            print("\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
            speed = (time.time() - time_now) / iter_count
            left_time = speed * ((args.train_epochs - epoch) * train_steps - i)
            print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
            iter_count = 0
            time_now = time.time()

        # if args.use_amp:
        #     scaler.scale(loss).backward()
        #     scaler.step(optim)
        #     scaler.update()
        else:
            loss.backward()
            optim.step()

In [11]:
def validation(args, vali_data, vali_loader, criterion, model, device):
        total_loss = []
        model.eval()
        with torch.no_grad():
            for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(vali_loader):
                batch_x = batch_x.float().to(device)
                batch_y = batch_y.float()

                batch_x_mark = batch_x_mark.float().to(device)
                batch_y_mark = batch_y_mark.float().to(device)

                # decoder input
                dec_inp = torch.zeros_like(batch_y[:, -args.pred_len:, :]).float()
                dec_inp = torch.cat([batch_y[:, :args.label_len, :], dec_inp], dim=1).float().to(device)
                # encoder - decoder
                if args.use_amp:
                    with torch.cuda.amp.autocast():
                        if args.output_attention:
                            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                        else:
                            outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                else:
                    if args.output_attention:
                        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                    else:
                        outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)
                f_dim = -1 if args.features == 'MS' else 0
                batch_y = batch_y[:, -args.pred_len:, f_dim:].to(device)

                pred = outputs.detach().cpu()
                true = batch_y.detach().cpu()

                loss = criterion(pred, true)

                total_loss.append(loss)
        total_loss = np.average(total_loss)
        model.train()
        return total_loss


In [12]:
device = torch.device(f"cuda:{args.gpu}" if args.use_gpu else "cpu")
model = FEDformer.Model(args).to(device)
model_optim = optim.Adam(model.parameters(), lr=args.learning_rate)
criterion = nn.MSELoss()

fourier enhanced block used!
modes=64, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
fourier enhanced block used!
modes=64, index=[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 29, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
 fourier enhanced cross attention used!
modes_q=64, index_q=[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71]
modes_kv=48, index_kv=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39

In [13]:
from sklearn.model_selection import train_test_split

def split_data(data, val_ratio=0.2, random_seed=42):
    """
    Splits a list of data into training and validation datasets.

    Args:
        data (list): The complete dataset to split.
        val_ratio (float): The proportion of data to use for validation (default: 0.2).
        random_seed (int): Random seed for reproducibility (default: 42).

    Returns:
        tuple: Two lists - training data and validation data.
    """
    train_data, val_data = train_test_split(data, test_size=val_ratio, random_state=random_seed)
    return train_data, val_data

In [14]:
train_data, val_data = split_data(fdata)

In [15]:
# data = [torch.as_tensor(cycle.astype(float).values) for cycle in cycles]
# train_data, val_data = split_data(data, val_ratio=0.2)
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size=args.batch_size, shuffle=False)

In [16]:
for epoch in range(args.train_epochs):
    epoch_time = time.time()
    train_steps = len(train_loader)
    iter_count = 0
    train(args, train_loader, model, model_optim, device, criterion, epoch, train_steps, iter_count)
    print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
    train_loss = np.average(train_loss)
    vali_loss = validation(val_data, val_loader, criterion)
    # test_loss = validation(test_data, test_loader, criterion)

    print("Epoch: {0}, Steps: {1} | Train Loss: {2:.7f} Vali Loss: {3:.7f}".format(
        epoch + 1, train_steps, train_loss, vali_loss))

KeyboardInterrupt: 